In [16]:
# ライブラリ(パッケージ)のインポート
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
# 作成したモデルのネットワークをそのままコピペ
# torch.nn.Moduleによるネットワークモデルの定義
class Mnist_net(nn.Module):
    def __init__(self, INPUT_FEATURES=784, OUTPUT_RESULTS=10):
        # 定数（モデル定義時に必要となるもの）
        self.INPUT_FEATURES = INPUT_FEATURES# 入力（特徴）の数： 2
        self.LAYER1_NEURONS = 256      # ニューロンの数： 256
        self.LAYER2_NEURONS = 128      # ニューロンの数： 128
        self.OUTPUT_RESULTS = OUTPUT_RESULTS# 出力結果の数： 10

        super(Mnist_net, self).__init__()
        # 入力層→中間層①：1つ目のレイヤー（layer）
        self.layer1 = nn.Linear(
            self.INPUT_FEATURES,                # 入力ユニット数：784
            self.LAYER1_NEURONS)                # 次のレイヤーの出力ユニット数：256

        # 中間層①→中間層②：2つ目のレイヤー（layer）
        self.layer2 = nn.Linear(
            self.LAYER1_NEURONS,                # 入力ユニット数：256
            self.LAYER2_NEURONS)                # 次のレイヤーへの出力ユニット数：128

        # 中間層②→出力層：2つ目のレイヤー（layer）
        self.layer_out = nn.Linear(
            self.LAYER2_NEURONS,                # 入力ユニット数：128
            self.OUTPUT_RESULTS)                # 出力結果への出力ユニット数：10

    def forward(self, x):
        # フィードフォワードを定義
        # 「出力＝活性化関数（第n層（入力））」の形式で記述する
        x = x.view(-1, 784) # 28×28の配列を784の一次元配列に変換
        x = F.relu(self.layer1(x))  # 活性化関数はreluとして定義
        x = F.relu(self.layer2(x))  # 同上 
        x = F.softmax(self.layer_out(x), dim=1)  # 出力する際はsoftmaxを使用
        return x

In [18]:

"""
Get_mnist_data関数
> Mnistのデータセットをダウンロードしてくる
batch_size          # バッチサイズ
"""
def Get_mnist_data(batch_size):
    # Totenser関数でtensor化
    # Normalise関数で正規化(-1.0〜1.0をとるような値として変換)
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])
    
    # 変数datasetにMNISTのデータセットを格納
    # ダウンロードしたデータセットは、./dataフォルダに配置される。
    # 訓練用のデータセットを変数train_datasetに格納
    train_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform, target_transform=None)
    
    # 学習する必要がないので、DataLoaderを用いてバッチサイズにまとめる処理は行わない
    # trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    # テスト(検証)用のデータセットを変数test_datasetに格納
    test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform, target_transform=None)
    
    # 学習する必要がないので、DataLoaderを用いてバッチサイズにまとめる処理は行わない
    # testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    # ダウンロードした訓練用データとテスト(検証)用データを保存
    return train_dataset, test_dataset


In [19]:
# 学習モデルを定義
model = Mnist_net() # 変数modelにMnist_netクラスのネットワークモデルを定義

# 変数modelに保存した学習したモデルのパラメータを読み込み
model.load_state_dict(torch.load("./mnist_model")) 

# 読み込んだモデルを検証モードにする
model.eval()

Mnist_net(
  (layer1): Linear(in_features=784, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=10, bias=True)
)

In [20]:
batch_size = 128 # バッチサイズ: 128

# 変数loader_trainとloader_validに学習用データとテスト用データをダウンロード
loader_train, loader_valid = Get_mnist_data(batch_size)

# 変数valid_Xにテスト用データの一番目の28×28の画像データの値を格納
valid_X = loader_valid[0][0]

# 変数valid_yにテスト用データの一番目の28×28の画像データの正解ラベルを格納
valid_y = loader_valid[0][1]

In [22]:
# 読み込んだモデルに変数valid_Xに格納された28×28の画像データの値を入力し、変数pred_yに出力
pred_y = model(valid_X)

# 変数pred_yは10個の値(onehotラベル)になっているので、0〜9のどの数字の手書き文字かを表現するラベルに変換する
_, pred_y = torch.max(pred_y.data, 1)

# 変数pred_yをnumpy型に変換
pred_y = pred_y.numpy()

# 学習済みのモデルに28×28の画像データを入力して、どの数字の手書き文字かを判別した結果を表示
print("model out : {}".format(pred_y))

# 実際の正解ラベルを表示
print("answer : {}".format(valid_y))

model out : [7]
answer : 7
